In [40]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver 
import json
import csv
import time

In [2]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")
browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

/Users/joshfeldman/Harvard/Courses/Fall2019/Capstone/AC297r_2019_somerville_parking/somerville/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
def clean_address(site_addr):
    num = str(row.ADDR_NUM).split('-')[0]
    addr = row.FULL_STR.lower()
    clean_addr = f"{num} {addr}, somerville, ma"
    clean_addr = (clean_addr
                  .replace('  ', ' ')
                  .replace(' ', '%20')
                  .replace(',','%2C'))
    return clean_addr

def get_unit_url(addr):
    # prepare url
    addr_url = (
        f"https://www.redfin.com/stingray/do/location-autocomplete?location={addr}"
        "&start=0&count=10&v=2&market=boston&al=1&iss=false&ooa=true&boostParams=1_1_8693141&mrs=false"
    )
    
    # make API request
    browser.get(addr_url)
    time.sleep(1)
    
    # parse request
    html = browser.page_source
    soup_search = BeautifulSoup(html)
    data = json.loads(soup_search.body.string.replace('{}&&',''))
    
    # get unit page URL
    redfin_url = 'https://www.redfin.com'
    url_path = data['payload']['sections'][0]['rows'][0]['url']
    full_url = redfin_url + url_path
    return full_url

def get_parking_stats(full_url, mbl):
    # load page 
    browser.get(full_url)
    time.sleep(1)
    soup_parcel = BeautifulSoup(browser.page_source)
    
    # parse page
    clean_stats = []
    try:
        parking_info = soup_parcel.body.find('h3', string='Parking Information').parent.parent
        parking_stats = parking_info.findAll(class_ = "entryItemContent")
        
        for stat in parking_stats:
            value = stat.span.text.strip()
            field_name = (stat.text.replace(value, '')
                          .replace(':','')
                          .strip())
            clean_stat = {'field_name': field_name, 
                          'value': value, 
                          'mbl': mbl}
            clean_stats.append(clean_stat)
        
        clean_stats.append({'field_name': 'has_data', 
                          'value': True, 
                          'mbl': mbl})
    except AttributeError:
        clean_stats.append({'field_name': 'has_data', 
                          'value': False, 
                          'mbl': mbl})
    return clean_stats

def save_stats(clean_stats, csv_file = "../data/redfin_data.csv"):
    import csv
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("I/O error")

In [55]:
df = pd.read_excel(
    '../data/residence_addresses_googlestreetview.xlsx',
    sheet_name='residential by unit'
)

In [60]:
df_tabular = pd.read_csv(
    '../data/residence_addresses_googlestreetview_clean.csv'
)
df_tabular.MBL

0         1-C-5
1         1-C-6
2         1-C-7
3         1-C-8
4         1-C-9
          ...  
11699    97-G-8
11700    97-H-3
11701    97-I-1
11702    97-I-2
11703    99-A-9
Name: MBL, Length: 11704, dtype: object

In [54]:
for addr in df.SITE_ADDR:
    print(addr)

190 BOSTON AVE, Somerville, MA
188 BOSTON AVE, Somerville, MA
184 BOSTON AVE, Somerville, MA
180 BOSTON AVE, Somerville, MA
178 BOSTON AVE, Somerville, MA
193 BOSTON AVE, Somerville, MA
7 STOUGHTON ST, Somerville, MA
157 BOSTON AVE, Somerville, MA
42 IRVINGTON RD, Somerville, MA
40 IRVINGTON RD, Somerville, MA
36 IRVINGTON RD, Somerville, MA
30 IRVINGTON RD, Somerville, MA
26-28 IRVINGTON RD, Somerville, MA
24 IRVINGTON RD, Somerville, MA
18 IRVINGTON RD, Somerville, MA
14 IRVINGTON RD, Somerville, MA
10 IRVINGTON RD, Somerville, MA
519 MYSTIC VALLEY PKWY, Somerville, MA
159 BOSTON AVE, Somerville, MA
523 MYSTIC VALLEY PKWY, Somerville, MA
527 MYSTIC VALLEY PKWY, Somerville, MA
533 MYSTIC VALLEY PKWY, Somerville, MA
165 BOSTON AVE, Somerville, MA
167-169 BOSTON AVE, Somerville, MA
171 BOSTON AVE, Somerville, MA
177 BOSTON AVE, Somerville, MA
181 BOSTON AVE, Somerville, MA
183 BOSTON AVE, Somerville, MA
187-189 BOSTON AVE, Somerville, MA
80 PROFESSORS ROW, Somerville, MA
17 LATIN WAY, S

99 BOSTON AVE, Somerville, MA
95-97 BOSTON AVE, Somerville, MA
93 BOSTON AVE, Somerville, MA
91 BOSTON AVE, Somerville, MA
89-89 A BOSTON AVE, Somerville, MA
87 BOSTON AVE, Somerville, MA
270 CEDAR ST, Somerville, MA
252 CEDAR ST, Somerville, MA
248 CEDAR ST, Somerville, MA
143 BOSTON AVE, Somerville, MA
244 CEDAR ST, Somerville, MA
240 CEDAR ST, Somerville, MA
236 CEDAR ST, Somerville, MA
232-234 CEDAR ST, Somerville, MA
228 CEDAR ST, Somerville, MA
53 BOSTON AVE, Somerville, MA
57 BOSTON AVE, Somerville, MA
65 BOSTON AVE, Somerville, MA
69 BOSTON AVE, Somerville, MA
71 BOSTON AVE, Somerville, MA
137-139 BOSTON AVE, Somerville, MA
77 BOSTON AVE, Somerville, MA
79 BOSTON AVE, Somerville, MA
83 BOSTON AVE, Somerville, MA
85 BOSTON AVE, Somerville, MA
135 BOSTON AVE, Somerville, MA
133 BOSTON AVE, Somerville, MA
129 BOSTON AVE, Somerville, MA
125 BOSTON AVE, Somerville, MA
121-121 BOSTON AVE, Somerville, MA
94 BOSTON AVE, Somerville, MA
54 BOSTON AVE, Somerville, MA
59 PRICHARD AVE, Some

277 ALEWIFE BROOK PKWY, Somerville, MA
279 ALEWIFE BROOK PKWY, Somerville, MA
281 ALEWIFE BROOK PKWY, Somerville, MA
283 ALEWIFE BROOK PKWY, Somerville, MA
285 ALEWIFE BROOK PKWY, Somerville, MA
46 HIGH ST, Somerville, MA
287 ALEWIFE BROOK PKWY, Somerville, MA
291 ALEWIFE BROOK PKWY, Somerville, MA
293 ALEWIFE BROOK PKWY, Somerville, MA
295 ALEWIFE BROOK PKWY, Somerville, MA
297 ALEWIFE BROOK PKWY, Somerville, MA
299 ALEWIFE BROOK PKWY, Somerville, MA
303 ALEWIFE BROOK PKWY, Somerville, MA
305 ALEWIFE BROOK PKWY, Somerville, MA
309 ALEWIFE BROOK PKWY, Somerville, MA
315 ALEWIFE BROOK PKWY, Somerville, MA
42 HIGH ST, Somerville, MA
38 HIGH ST, Somerville, MA
34 HIGH ST, Somerville, MA
30 HIGH ST, Somerville, MA
26 HIGH ST, Somerville, MA
497 BROADWAY, Somerville, MA
477 BROADWAY, Somerville, MA
463 BROADWAY, Somerville, MA
428 BROADWAY, Somerville, MA
40 ADAMS ST, Somerville, MA
36 ADAMS ST, Somerville, MA
30 ADAMS ST, Somerville, MA
24-26 ADAMS ST, Somerville, MA
18 ADAMS ST, Somervill

11-15 FENWICK ST, Somerville, MA
19 FENWICK ST, Somerville, MA
54 HEATH ST, Somerville, MA
48 HEATH ST, Somerville, MA
34 HEATH ST, Somerville, MA
26 HEATH ST, Somerville, MA
22 HEATH ST, Somerville, MA
18 HEATH ST, Somerville, MA
80 HEATH ST, Somerville, MA
371 BROADWAY, Somerville, MA
373 BROADWAY, Somerville, MA
375 BROADWAY, Somerville, MA
5 BOND ST, Somerville, MA
9 BOND ST, Somerville, MA
15 BOND ST, Somerville, MA
17 BOND ST, Somerville, MA
74 HEATH ST, Somerville, MA
26 FENWICK ST, Somerville, MA
22 FENWICK ST, Somerville, MA
18 FENWICK ST, Somerville, MA
12 FENWICK ST, Somerville, MA
10 FENWICK ST, Somerville, MA
367 BROADWAY, Somerville, MA
369-369 BROADWAY, Somerville, MA
358 BROADWAY, Somerville, MA
48 DARTMOUTH ST, Somerville, MA
46 DARTMOUTH ST, Somerville, MA
42 DARTMOUTH ST, Somerville, MA
40-40 DARTMOUTH ST, Somerville, MA
38 DARTMOUTH ST, Somerville, MA
43 EVERGREEN AVE, Somerville, MA
67 THURSTON ST, Somerville, MA
71 THURSTON ST, Somerville, MA
75 THURSTON ST, Somer

22 DIMICK ST, Somerville, MA
90 CONCORD AVE, Somerville, MA
96 CONCORD AVE, Somerville, MA
94 CONCORD AVE, Somerville, MA
92 CONCORD AVE, Somerville, MA
5 LEON ST, Somerville, MA
9 LEON ST, Somerville, MA
27 DICKINSON ST, Somerville, MA
31 DICKINSON ST, Somerville, MA
33 DICKINSON ST, Somerville, MA
12 HAMMOND ST, Somerville, MA
8 HAMMOND ST, Somerville, MA
6 HAMMOND ST, Somerville, MA
88 CONCORD AVE, Somerville, MA
62 SPRINGFIELD ST, Somerville, MA
58 SPRINGFIELD ST, Somerville, MA
54 SPRINGFIELD ST, Somerville, MA
52 SPRINGFIELD ST, Somerville, MA
50 SPRINGFIELD ST, Somerville, MA
7 DICKINSON ST, Somerville, MA
9 DICKINSON ST, Somerville, MA
11 DICKINSON ST, Somerville, MA
13 DICKINSON ST, Somerville, MA
10 LEON ST, Somerville, MA
86 CONCORD AVE, Somerville, MA
8 LEON ST, Somerville, MA
84 CONCORD AVE, Somerville, MA
78 CONCORD AVE, Somerville, MA
74 CONCORD AVE, Somerville, MA
72 CONCORD AVE, Somerville, MA
70 CONCORD AVE, Somerville, MA
68 CONCORD AVE, Somerville, MA
66 SPRINGFIELD

In [42]:
df['url_friendly_addr'] = df.SITE_ADDR.apply(clean_address)

In [43]:
csv_file = "../data/redfin_data.csv"
csv_columns = ['field_name','value','mbl']
with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()

In [35]:
for row_id, row in df.iterrows():
    addr = row.url_friendly_addr
    mbl = row.MBL
    full_url = get_unit_url(addr)
    stats = get_parking_stats(full_url)

    break

[{'field_name': 'Parking Features', 'value': 'Tandem'}, {'field_name': '# of Parking Spaces', 'value': '2'}, {'field_name': 'Parking', 'value': 'Tandem Parking'}]


In [27]:
addr = url_friendly_addr[2]

In [28]:
full_url = get_unit_url(addr)
get_parking_stats(full_url)

[{'field_name': '# of Parking Spaces', 'value': '1'}]

In [169]:
addr = df.apply(clean_address, axis = 1)[1]
addr_url = (f"https://www.redfin.com/stingray/do/location-autocomplete?location={addr}"
"&start=0&count=10&v=2&market=boston&al=1&iss=false&ooa=true&boostParams=1_1_8693141&mrs=false")

In [30]:
df.loc[2]

ID                         17978
PROP_ID            90||C||25||M|
BLDG_VAL                       0
LAND_VAL                       0
OTHER_VAL                      0
TOTAL_VAL                      0
FY                          2020
LOT_SIZE                       0
LS_DATE                      NaT
LS_PRICE                     NaN
USE_CODE                     995
SITE_ADDR      17-19 A BONAIR ST
ADDR_NUM                   17-19
FULL_STR             A BONAIR ST
LOCATION                     NaN
SITE_CITY             SOMERVILLE
SITE_ZIP                    2145
OWNER1                       NaN
OWNER2                       NaN
OWN_ADDR1                    NaN
OWN_ADDR2                    NaN
OWN_CITY                     NaN
OWN_STATE                    NaN
OWN_ZIP                      NaN
OWN_CO                       NaN
LS_BOOK                      NaN
LS_PAGE                      NaN
REG_ID                       NaN
ZONE                         NaN
YEAR_BUILT                  1890
BLD_AREA  

In [170]:
dfbrowser.get(addr_url)
html = browser.page_source

In [171]:
soup_search = BeautifulSoup(html)

In [172]:
data = json.loads(soup_search.body.string.replace('{}&&',''))

In [173]:
redfin_url = 'https://www.redfin.com'
url_path = data['payload']['sections'][0]['rows'][0]['url']
full_url = redfin_url + url_path

In [174]:
browser.get(full_url)

In [175]:
soup_parcel = BeautifulSoup(browser.page_source)

In [176]:
parking_info = soup_parcel.body.find('h3', string='Parking Information').parent.parent
parking_stats = parking_info.findAll(class_ = "entryItemContent")

for stat in parking_stats:
    entry = stat.span.text.strip()
    field_name = stat.text.replace(entry, '').replace(':','').strip()
    print(field_name)
    print(entry)

Parking Features
Off-Street, Deeded
# of Parking Spaces
4
# of Garage Spaces
1
Parking
Off-Street Parking, Deeded
Garage
Attached Garage
